In [ ]:
#Import Required Libraries
import os
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define data directories
wikiart_dir = '/content/drive/MyDrive/DeepLearningProject/wikiart'
artemis_data_file = '/content/drive/MyDrive/DeepLearningProject/ProcessedData/artemis_preprocessed.csv'

# Load the ArtEmis data
artemis_data = pd.read_csv(artemis_data_file)

# Define emotion label encoder
le = LabelEncoder()
le.fit(artemis_data['emotion'])

# Define image size
img_size = (224, 224)


import tensorflow as tf

def load_image(file_path):
    # Read the image from the file_path using TensorFlow's io.read_file and image.decode_image functions
    image = tf.io.read_file(file_path)
    image = tf.image.decode_image(image)

    # Resize the image to the desired size (img_size) using TensorFlow's image.resize function
    image = tf.image.resize(image, img_size)

    # Convert the color space of the image from BGR (default in OpenCV) to RGB using TensorFlow's image.rgb_to_grayscale function
    image = tf.image.rgb_to_grayscale(image)

    # Normalize the pixel values of the image by converting the data type to float32 and dividing by 255.0
    # This brings the pixel values to the range [0, 1] which is commonly used in deep learning models
    image = tf.cast(image, tf.float32) / 255.0

    # Return the preprocessed image
    return image



The code starts by defining the paths to the data directories and files, specifically the directory containing the WikiArt images (wikiart_dir) and the file path to the preprocessed ArtEmis data (artemis_data_file). The ArtEmis data is loaded into a pandas DataFrame using the read_csv function. The data is read from the CSV file specified by artemis_data_file.

The code defines a label encoder (le) to encode the emotion labels present in the ArtEmis data. The encoder is fit on the 'emotion' column of the ArtEmis DataFrame using the fit method. The image size is defined as (224, 224), indicating the desired dimensions for the images after resizing. TensorFlow library is imported as tf.

The code defines a function called load_image that takes a file path as input and performs image preprocessing using TensorFlow functions. Inside the load_image function, the image is read from the file path using TensorFlow's io.read_file function, which reads the image data as a byte string. The decode_image function from TensorFlow's image module is used to decode the image from the byte string format.

The image is resized to the desired dimensions (img_size) using TensorFlow's image.resize function. This ensures that all images have the same size for consistency during model training. The color space of the image is converted from the default BGR (commonly used in OpenCV) to grayscale using TensorFlow's image.rgb_to_grayscale function. This converts the image to a single-channel grayscale representation.

The pixel values of the image are normalized by converting the data type to float32 and dividing by 255.0. This normalization step scales the pixel values to the range of [0, 1], which is a common practice in deep learning models. The preprocessed image is returned as the output of the load_image function.

In [ ]:
def generate_numpy_arrays(dataframe):
    # Get the total number of samples in the input dataframe
    num_samples = len(dataframe)

    # Initialize the data array with zeros
    data = np.zeros((num_samples, img_size[0], img_size[1], 3), dtype=np.float32)

    # Convert emotion labels to one-hot-encoded labels
    labels = to_categorical(le.transform(dataframe['emotion']), num_classes=len(le.classes_))

    # Define a helper function to load and preprocess an image
    def process_image(index):
        row = dataframe.iloc[index]
        image_path = os.path.join(wikiart_dir, row['art_style'], row['painting'] + '.jpg')
        return load_image(image_path)

    # Load and preprocess images in parallel
    images = Parallel(n_jobs=-1)(delayed(process_image)(i) for i in tqdm(range(num_samples), desc='Loading Images'))

    # Store the preprocessed images in the data array
    data[:] = images

    # Convert labels to float32
    labels = labels.astype(np.float32)

    # Return the data and labels arrays
    return data, labels



The provided code defines a function called generate_numpy_arrays that processes a DataFrame containing image data and generates numpy arrays for the images and their corresponding labels. Let's understand how the code works:

The function takes a DataFrame (dataframe) as input, which contains image data and emotion labels.  The total number of samples in the input DataFrame is obtained using the len function, and it is stored in the num_samples variable.

An array called data is initialized with zeros. It has dimensions (num_samples, img_size[0], img_size[1], 3) and a data type of float32. This array will hold the preprocessed image data.

The emotion labels in the DataFrame are transformed into one-hot-encoded labels using the to_categorical function. The labels are first transformed using the label encoder (le) to convert them into numeric representations. The number of classes is determined by the length of the label encoder's classes (len(le.classes_)).

The code defines a helper function, process_image, which takes an index as input and retrieves the corresponding row from the DataFrame. It constructs the file path for the image by joining the directory path (wikiart_dir), art style, and painting name with the appropriate file extension. This function then calls the load_image function (defined earlier) to preprocess the image.

The Parallel class from the joblib library is used to load and preprocess the images in parallel. The Parallel class runs multiple instances of the process_image function concurrently, speeding up the image loading process. The n_jobs=-1 parameter specifies that all available CPU cores should be used for parallel processing.

The images list is populated with the preprocessed images obtained from parallel processing. The tqdm function is used to display a progress bar indicating the loading progress. The preprocessed images in the images list are stored in the data array using the slicing operation data[:] = images. This copies the elements of the images list to the corresponding positions in the data array.

The data type of the labels is converted to float32 using the astype method.Finally, the function returns the data array (containing the preprocessed images) and the labels array (containing the corresponding labels) as the output.

train_df: 72% of the data for training (0.8 * 0.9 = 0.72)

val_df: 8% of the data for validation (0.8 * 0.1 = 0.08)

test_df: 20% of the data for testing

In [ ]:
train_df, test_df = train_test_split(artemis_data, test_size=0.2, random_state=2021)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2021)

# Find Number of samples used:
num_train_samples = len(train_df)
num_val_samples = len(val_df)
num_test_samples = len(test_df)

print(f"Number of training samples: {num_train_samples}")
print(f"Number of validation samples: {num_val_samples}")
print(f"Number of test samples: {num_test_samples} \n")

# Reduce the datasamples to train on colab:

#create reduction ratios 
train_frac = 0.05
val_frac = 0.1
test_frac = 0.1

#Create the reduced datasets
train_df_reduced = train_df.sample(frac=train_frac, random_state=2021)
val_df_reduced = val_df.sample(frac=val_frac, random_state=2021)
test_df_reduced = test_df.sample(frac=test_frac, random_state=2021)

num_train_samplesr = len(train_df_reduced)
num_val_samplesr = len(val_df_reduced)
num_test_samplesr =len(test_df_reduced)

print(f"Number of reduced training samples: {num_train_samplesr}")
print(f"Number of reduced validation samples: {num_val_samplesr}")
print(f"Number of reduced test samples: {num_test_samplesr}")



Number of training samples: 327372
Number of validation samples: 36375
Number of test samples: 90937 

Number of reduced training samples: 16369
Number of reduced validation samples: 3638
Number of reduced test samples: 9094


In [ ]:
# Generate the NumPy arrays for each dataset
train_data, train_labels = generate_numpy_arrays(train_df_reduced)
val_data, val_labels = generate_numpy_arrays(val_df_reduced)
test_data, test_labels = generate_numpy_arrays(test_df_reduced)

Loading Images:   5%|▌         | 888/16369 [02:15<20:45, 12.43it/s]/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Loading Images:  50%|████▉     | 1812/3638 [00:31<00:33, 54.68it/s]/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Loading Images:  12%|█▏        | 1080/9094 [00:19<02:24, 55.37it/s]/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Loading Images: 100%|██████████| 9094/9094 [04:52<00:00

In [ ]:
# Save the NumPy arrays to Google Drive
save_dir = '/content/drive/MyDrive/DeepLearningProject/GeneratedData5p'

np.save(save_dir + '/train_data.npy', train_data)
np.save(save_dir + '/train_labels.npy', train_labels)

# Save the arrays as compressed NumPy files
np.savez_compressed(save_dir + '/train_data.npz', data=train_data)
np.savez_compressed(save_dir + '/train_labels.npz', labels=train_labels)
# Repeat for other arrays


np.save(save_dir + '/val_data.npy', val_data)
np.save(save_dir + '/val_labels.npy', val_labels)

# Save the arrays as compressed NumPy files
np.savez_compressed(save_dir + '/val_data.npz', data=train_data)
np.savez_compressed(save_dir + '/val_labels.npz', labels=train_labels)

np.save(save_dir + '/test_data.npy', test_data)
np.save(save_dir + '/test_labels.npy', test_labels)

# Save the arrays as compressed NumPy files
np.savez_compressed(save_dir + '/test_data.npz', data=train_data)
np.savez_compressed(save_dir + '/test_labels.npz', labels=train_labels)
